# Loading test data

## DOC
- [Introduction to the Python Driver for Oracle Database](https://python-oracledb.readthedocs.io/en/latest/user_guide/introduction.html#introduction-to-the-python-driver-for-oracle-database)
- [Python python-oracledb Driver](https://oracle.github.io/python-oracledb/)
- [Python: Read Data from Oracle Database](https://kontext.tech/article/1019/python-read-data-from-oracle-database)

## Connecting to the database via the ORACLE "thin" client

In [1]:
    import oracledb
    import getpass
    
    username = "CUSTDOC"
    userpwd = getpass.getpass("Enter password: ")

    host="localhost"
    port="1521"
    service_name="XEPDB1"
    dsn = f'{username}/{userpwd}@{host}:{port}/{service_name}'
   

Enter password:  ········


## Preparing test data

In [5]:
import oracledb
import os
from faker import Faker

def next_custid(numcust):
    """
    Generating CUSTID value. numcust - integer"
    Генерація значення CUSTID. numcust - ціле число
    """
    FIXED_LENGTH = 9
    # 1. Convert int на str
    number_str = str(numcust) 
    # 2. use zfill() to add leading zeros
    result_zfill = number_str.zfill(FIXED_LENGTH)
    return result_zfill

def insert_to_doc( CUSTID, TYPEDOC, ISACRUAL):
    """INSERT CUSTOMER DOCUMET TYPE"""
    try:
        sql_insert_doc = """ 
            INSERT INTO CUSTDOC.CUST$DOCS
            (CUSTID, TYPEDOC, ISACRUAL )
            VALUES
            (:CUSTID, :TYPEDOC, :ISACRUAL)
             RETURNING IDDOC INTO :returned_iddoc 
        """       
        returned_iddoc = cursor.var(oracledb.NUMBER)
        cursor.execute(sql_insert_doc, {
            'CUSTID': CUSTID,
            'TYPEDOC': TYPEDOC,
            'ISACRUAL': ISACRUAL, 
            'returned_iddoc': returned_iddoc
        })

        return returned_iddoc.getvalue()[0]
    except oracledb.Error as e:
        error_obj, = e.args
        print(f"Oracle Error: {error_obj.code} - {error_obj.message}")
    except Exception as error:
        print(f"Unexpected error: {type(error).__name__}")
        print(f"Error details: {error}")
        print(sys.exc_info())          


def insert_to_blob( custid, iddoc, file_path,  file_desc):
    """Read a file and load it into a BLOB field of an ORACLE table"""
    try:
        with open(file_path, 'rb') as f:
                file_data = f.read()
    
        file_name = os.path.basename(file_path)
        sql_insert_blob = """
            INSERT INTO CUSTDOC.CUST$DOCS$ATTACH 
            (IDDOC, FILE_NAME, FILE_DESC, FILE_CONTENT)
            VALUES 
            (:returned_iddoc, :file_name,  :file_desc, :file_content)
            RETURNING IDFL INTO :returned_idfl """
        returned_idfl = cursor.var(oracledb.NUMBER)
        cursor.execute(sql_insert_blob, {
            'returned_iddoc': iddoc,
            'file_name': 'CU_' + custid + '_' + file_name,
            'file_desc': file_desc,
            'file_content': file_data, 
            'returned_idfl': returned_idfl
        })
        return returned_idfl.getvalue()[0]
    except oracledb.Error as e:
        error_obj, = e.args
        print(f"Oracle Error: {error_obj.code} - {error_obj.message}")
    except Exception as error:
        print(f"Unexpected Error: {type(error).__name__}")
        print(f"Error Details: {error}")
        print(sys.exc_info())          
            
#=============================================================
try:
    # Connect to DB
    connection = oracledb.connect(dsn)
    cursor = connection.cursor()
    sqld="""DELETE FROM CUSTDOC.CUST$CUST"""
    cursor.execute(sqld) 
    connection.commit();
    
    sql = """insert into CUSTDOC.CUST$CUST 
              ( CUSTID, CUSTTYPE, CUSTNAME ) VALUES 
              ( :A_CUSTID, :A_CUSTTYPE, :A_CUSTNAME )"""

    custid_int=0
    fake_en = Faker('en_US') 
    print("--- Individuals (English) ---")
    for _ in range(56):
        # Generate person name
        full_name = fake_en.name() 
       
        custid_int+=1
        custid=next_custid(custid_int) 
        #print(f"{full_name} for custi={custid}")
        cursor.execute(sql, 
                       {
                        "A_CUSTID": custid, 
                        "A_CUSTTYPE": "CI", 
                        "A_CUSTNAME": full_name 
                        }
         )

        if custid_int % 2 == 0:
            ret_iddoc=insert_to_doc(custid, 'FOR_PASS', 'Y')
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/IMG_20190512_123820.jpg',  'Document: IMG_20190512_123820.jpg')  
        elif  custid_int % 3 == 0:
            ret_iddoc=insert_to_doc(custid, 'NAT_ID', 'Y')
            
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/IMG_20190512_123820.jpg',  'Document:  IMG_20190512_123820.jpg')    
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/IMG_20190512_124107.jpg',  'Document:  IMG_20190512_124107.jpg')    
            
            ret_iddoc=insert_to_doc(custid, 'TAX_ID', 'Y')
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/IMG_20190515_135249.jpg',  'Document:  IMG_20190515_135249.jpg')    
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/IMG_20190512_124206.jpg',  'Document:  IMG_20190512_124206.jpg')    
        elif  custid_int % 5 == 0:
            ret_iddoc=insert_to_doc(custid, 'DRV_LIC', 'Y')
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/1000001465.jpg',  '1000001465.jpg')    
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/1000001462.jpg',  '1000001462.jpg') 
        else:    
            ret_iddoc=insert_to_doc(custid, 'NAT_ID', 'Y')
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/IMG_20190511_152919.jpg',  'Document:   IMG_20190511_152919.jpg')    
    connection.commit()
    print("--- Legal Ent (English) ---")
    for _ in range(56):
        # Generate company name
        company_name = fake_en.company()
        custid_int+=1
        custid=next_custid(custid_int) 
        #print(f"{company_name} for custi={custid}")
        cursor.execute(sql, 
                       {
                        "A_CUSTID": custid, 
                        "A_CUSTTYPE": "LE", 
                        "A_CUSTNAME": company_name 
                        }
         )

        if custid_int % 2 == 0:
            ret_iddoc=insert_to_doc(custid, 'AoA', 'Y')
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/IMG_20190512_123820.jpg',  'Document: IMG_20190512_123820.jpg')  
        elif  custid_int % 3 == 0:
            ret_iddoc=insert_to_doc(custid, 'REG_EXT', 'Y')
            
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/IMG_20190512_123820.jpg',  'Document:  IMG_20190512_123820.jpg')    
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/IMG_20190512_124107.jpg',  'Document:  IMG_20190512_124107.jpg')    
            
            ret_iddoc=insert_to_doc(custid, 'COI', 'Y')
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/IMG_20190515_135249.jpg',  'Document:  IMG_20190515_135249.jpg')    
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/IMG_20190512_124206.jpg',  'Document:  IMG_20190512_124206.jpg')    
        elif  custid_int % 5 == 0:
            ret_iddoc=insert_to_doc(custid, 'PoA', 'Y')
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/1000001465.jpg',  '1000001465.jpg')    
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/1000001462.jpg',  '1000001462.jpg') 
        else:    
            ret_iddoc=insert_to_doc(custid, 'REG_EXT', 'Y')
            ret_idfl=insert_to_blob(custid, ret_iddoc, './data/IMG_20190511_152919.jpg',  'Document:   IMG_20190511_152919.jpg')            
    connection.commit()
except oracledb.Error as e:
    error_obj, = e.args
    print(f"Oracle Error: {error_obj.code} - {error_obj.message}")
except Exception as error:
    print(f"Unexpected Error: {type(error).__name__}")
    print(f"Error details: {error}")
    print(sys.exc_info())    
finally:
    if cursor:
        cursor.close()
        print("\nThe cursor is closed.")
    if connection:
        connection.close()
        print("The connection to the database has been closed.")

--- Individuals (English) ---
--- Legal Ent (English) ---

The cursor is closed.
The connection to the database has been closed.


#### Viewing test data

In [6]:
sql_query = """
    SELECT A.CUSTID, A.CUSTTYPE, A.CUSTNAME, B.TYPEDOC, B.ISACRUAL, F.FILE_NAME, B.IDDOC, F.IDFL
    FROM CUSTDOC.CUST$CUST  A
    LEFT OUTER JOIN CUSTDOC.CUST$DOCS B ON B.CUSTID=A.CUSTID
    LEFT OUTER JOIN CUSTDOC.CUST$DOCS$ATTACH  F ON F.IDDOC=B.IDDOC
 """
connection = None
cursor = None

try:
    connection = oracledb.connect(dsn=dsn)
    cursor = connection.cursor()
    cursor.execute(sql_query)
    column_names = [col[0] for col in cursor.description]
    
    print("-" * 130)
    # Header
    print(f"{column_names[0]:<15} {column_names[1]:<18} {column_names[2]:<20} {column_names[3]:<10} {column_names[4]:<8} {column_names[5]:<38} {column_names[6]:<5} {column_names[7]:<5}")
    print("-" * 130)
    # 4. Get Rows
    rows = cursor.fetchall()
    # 5. Rows iteration
    if rows:
        for row in rows:
            # Print out rows
            # :<15 left sid 15 character field
            print(f"{row[0]:<15} {row[1]:<8} {row[2]:<30} {row[3]:<10} {row[4]:<8} {row[5]:<38} {row[6]:<5} {row[7]:<5}")
    else:
        print("The query did not return any rows.")
        
    print("-" * 130)
    print(f"Fetched rows={len(rows)}")

except oracledb.Error as e:
    # Обробка помилок Oracle
    error_obj, = e.args
    print(f"Oracle Error: {error_obj.code} - {error_obj.message}")
finally:
    # 6. Закриття курсора та з'єднання
    if cursor:
        cursor.close()
        print("\nThe cursor is closed.")
    if connection:
        connection.close()
        print("The connection to the database has been closed.")

SyntaxError: unterminated string literal (detected at line 45) (3503653694.py, line 45)